In [3]:
# Creates a playlist of more relevant recommended tracks of a playlist

# install Spotipy first
#!pip install Spotipy
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from spotipy import oauth2
import csv
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util


cid ='a611ab78dcf5445ea1d91bf605134cbd' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = 'c196412417624657a55c5f71c44fd3b9' # Client Secret; copy this from your app
username = 'hpa6zlnetl6hib5bnp1b5laul' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='https://www.google.com.tr/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

Couldn't read cache at: .cache-hpa6zlnetl6hib5bnp1b5laul


Enter the URL you were redirected to: https://www.google.com.tr/?code=AQDnGhCb0JZelhAHeZY2ZenrmKWHLnw7V4CEk94WWbL7uqD-z87xkugPrk3frKRyTQYjuFCADAwS9LW2iwfCoQjNYHjj4wqDNUbQPPv0XEvwx6sW00VzOs84ifG7kycQL2nr-AapP-JWgbR_RWc9DH33ISoQT3pTGo9misKVJxbOg2JiqvcWbW0QWbmoLlop7Powi8vYTnUewGzwX97WCumP5CDxf_e7pFOe18YCBzfihZqyJ3WBrBayYPC_jSv-


Couldn't read cache at: .cache-hpa6zlnetl6hib5bnp1b5laul


In [5]:
import pandas as pd

dataset_path = "data.csv"
with open(dataset_path, 'r', encoding="utf8") as token_file:
    lines = csv.reader(token_file)
    dataset = list(lines)
    data = np.array(dataset)
data = data[1:, :]
print(data.shape)
song_valence = data[:, 17]
song_acousticness = data[:, 0]
song_artists = data[:, 1]
song_danceability = data[:, 2]

song_duration_ms = data[:, 3]
song_energy = data[:, 4]
song_id = data[:, 6]
song_popularity = data[:, 13]

song_instrumentalness = data[:, 7]
song_key = data[:, 8]
song_liveness = data[:, 9]
song_loudness = data[:, 10]

song_mode = data[:, 11]
song_name = data[:, 12]
song_speechiness = data[:, 15]
song_tempo = data[:, 16]

song_info = {'name': [], 'id': [], 'artists': []}
features_dict = {'valence': [], 'popularity': [], 'acousticness': [], 'danceability': [], 'duration_ms': [], 'energy': [], 'instrumentalness': [], 'key': [], 'liveness': [], 'loudness': [], 'mode': [], 'speechiness': [], 'tempo': []}
for i in range(0, data.shape[0]):
    song_info['name'].append(song_name[i])
    song_info['id'].append(song_id[i])
    song_info['artists'].append(song_artists[i])
    features_dict['valence'].append(song_valence[i])
    features_dict['acousticness'].append(song_acousticness[i])
    features_dict['danceability'].append(song_danceability[i])
    features_dict['duration_ms'].append(song_duration_ms[i])
    features_dict['popularity'].append(song_popularity[i])
    features_dict['energy'].append(song_energy[i])
    features_dict['instrumentalness'].append(song_instrumentalness[i])
    features_dict['key'].append(song_key[i])
    features_dict['liveness'].append(song_liveness[i])
    features_dict['loudness'].append(song_loudness[i])
    features_dict['mode'].append(song_mode[i])
    features_dict['speechiness'].append(song_speechiness[i])
    features_dict['tempo'].append(song_tempo[i])
playlist_df = pd.DataFrame(features_dict, index = song_info['name'])
playlist_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
#Create a dataframe of your playlist including tracks' names and audio features
sourcePlaylistID = '5TplGaik7PSSBqH6eRRE8E'
sourcePlaylist = sp.user_playlist(username, sourcePlaylistID);
tracks = sourcePlaylist["tracks"];
songs = tracks["items"];

track_ids = []
track_names = []

for i in range(0, len(songs)):
    if songs[i]['track']['id'] != None: # Removes the local tracks in your playlist if there is any
        track_ids.append(songs[i]['track']['id'])
        track_names.append(songs[i]['track']['name'])

features = []
for i in range(0,len(track_ids)):
    audio_features = sp.audio_features(track_ids[i])
    tr = sp.track(track_ids[i])['popularity']
    for track in audio_features:
        print(track)
        features.append(track)
#print(features)
playlist_df = pd.DataFrame(features, index = track_names)

In [ ]:
playlist_df.head()

In [ ]:
#playlist_df=playlist_df[["id", "acousticness", "danceability", "duration_ms", 
#                         "energy", "instrumentalness",  "key", "liveness",
#                         "loudness", "mode", "speechiness", "tempo", "valence"]]
#playlist_df.head()

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer

#v=TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 6), max_features=10000)
#X_names_sparse = v.fit_transform(track_names)
#X_names_sparse.shape

In [ ]:
## Change the below info for each playlist manually to get better results
# Give ratings to your tracks with respect to their playlist relevances
# Rate them from 1-10, give higher ratings to those tracks which you think best chracterizes your playlist
# If you order your playlist by relevance while creating it, this step will become easier
# So now, we will deal with a classification task

features_array = np.zeros((data.shape[0]+1, 11))

input_ = np.zeros((11))
input_[0] = playlist_df['valence'][0]
input_[1] = playlist_df['acousticness'][0]
input_[2] = playlist_df['danceability'][0]
input_[3] = playlist_df['popularity'][0]
input_[4] = playlist_df['energy'][0]
input_[5] = playlist_df['instrumentalness'][0]
input_[6] = playlist_df['liveness'][0]
input_[7] = playlist_df['loudness'][0]
input_[8] = playlist_df['mode'][0]
input_[9] = playlist_df['speechiness'][0]
input_[10] = playlist_df['tempo'][0]

features_array[data.shape[0], :] = input_

features_array[:data.shape[0],0] = np.array(features_dict['valence']).T
features_array[:data.shape[0],1] = np.array(features_dict['acousticness']).T
features_array[:data.shape[0],2] = np.array(features_dict['danceability']).T
features_array[:data.shape[0],3] = np.array(features_dict['popularity']).T
features_array[:data.shape[0],4] = np.array(features_dict['energy']).T
features_array[:data.shape[0],5] = np.array(features_dict['instrumentalness']).T
features_array[:data.shape[0],6] = np.array(features_dict['liveness']).T
features_array[:data.shape[0],7] = np.array(features_dict['loudness']).T
features_array[:data.shape[0],8] = np.array(features_dict['mode']).T
features_array[:data.shape[0],9] = np.array(features_dict['speechiness']).T
features_array[:data.shape[0],10] = np.array(features_dict['tempo']).T

features_array = features_array.astype('float64')

model = NearestNeighbors(n_neighbors = 1000, algorithm = 'ball_tree')

scalar = StandardScaler()
scalar.fit(features_array)
features_array = scalar.transform(features_array)
input_2 = features_array[data.shape[0], :]
features_array = features_array[:data.shape[0], :]
model.fit(features_array)

#scalar2 = StandardScaler()
#scalar2.fit([input_])
#input_2 = scalar2.transform([input_])


print(features_array.shape)

distances, indices = model.kneighbors([input_2])
recorded_indices = indices
print(distances[0, :10])
print(distances[0, 990:])

closest_1000_point = {'valence': [], 'popularity': [], 'mode': [], 'acousticness': [], 'danceability': [], 'energy': [], 'instrumentalness': [], 'liveness': [], 'loudness': [], 'speechiness': [], 'tempo': [], 'ratings': []}

for i in range(0, 1000):
    closest_1000_point['valence'].append(features_array[indices[0, i], 0])
    closest_1000_point['acousticness'].append(features_array[indices[0, i], 1])
    closest_1000_point['danceability'].append(features_array[indices[0, i], 2])
    closest_1000_point['popularity'].append(features_array[indices[0, i], 3])
    closest_1000_point['energy'].append(features_array[indices[0, i], 4])
    closest_1000_point['instrumentalness'].append(features_array[indices[0, i], 5])
    closest_1000_point['liveness'].append(features_array[indices[0, i], 6])
    closest_1000_point['loudness'].append(features_array[indices[0, i], 7])
    closest_1000_point['mode'].append(features_array[indices[0, i], 8])
    closest_1000_point['speechiness'].append(features_array[indices[0, i], 9])
    closest_1000_point['tempo'].append(features_array[indices[0, i], 10])
    
    rem = i // 100
    closest_1000_point['ratings'].append(10-rem)
#playlist_df['ratings']=[10, 9, 9, 10, 8, 6, 8, 6, 4, 3, 6, 9]
#playlist_df.head()
playlist_df = pd.DataFrame(closest_1000_point)
playlist_df.head()

In [ ]:
# Analyze feature importances
from sklearn.ensemble.forest import RandomForestRegressor, RandomForestClassifier

X_train = playlist_df.drop(['ratings'], axis=1)
y_train = playlist_df['ratings']
forest = RandomForestClassifier(random_state=42, max_depth=5, max_features=11) # Set by GridSearchCV below
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature rankings
print("Feature ranking:")
  
for f in range(len(importances)):
    print("%d. %s %f " % (f + 1, 
            X_train.columns[indices[f]], 
            importances[indices[f]]))


In [ ]:
# Apply pca to the scaled train set first

from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='white')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

X_scaled = StandardScaler().fit_transform(X_train)
print(X_scaled.shape)
pca = decomposition.PCA().fit(X_scaled)

plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.xlim(0, 12)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(8, c='b') # Tune this so that you obtain at least a 95% total variance explained
plt.axhline(0.95, c='r')
plt.show();

In [ ]:
# Fit your dataset to the optimal pca
pca1 = decomposition.PCA(n_components=8)
X_pca = pca1.fit_transform(X_scaled)
print(X_pca.shape)

In [ ]:
# You may want to check the results of TSNE also
#from sklearn.manifold import TSNE

#tsne = TSNE(random_state=17)
#X_tsne = tsne.fit_transform(X_scaled)
#print(X_tsne.shape)

In [ ]:
#from scipy.sparse import csr_matrix, hstack

#X_train_last = csr_matrix(hstack([X_pca, X_names_sparse])) # Check with X_tsne + X_names_sparse also

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

# Initialize a stratified split for the validation process
skf = StratifiedKFold(n_splits=5, shuffle=True)
for train_index, test_index in skf.split(X_train, y_train):
    # Decision Trees First
    X_train_cv = np.array(X_train)[train_index, :]
    X_test_cv = np.array(X_train)[test_index, :]
    y_train_cv = np.array(y_train)[train_index]
    y_test_cv = np.array(y_train)[test_index]
    #tree = DecisionTreeClassifier(criterion = "entropy", splitter = "random")
    tree = DecisionTreeClassifier()
    tree_params = {'max_depth': range(1,11), 'max_features': range(1,11)}

    tree_grid = GridSearchCV(tree, tree_params, n_jobs=-1, verbose=True)

    tree_grid.fit(X_train_cv, y_train_cv)
    print(tree_grid.best_estimator_)
    print(tree_grid.best_score_)


In [ ]:
# Random Forests second

parameters = {'max_features': range(1,11), 'min_samples_leaf': range(1,11), 'max_depth': range(1, 11)}
rfc = RandomForestClassifier(n_estimators=100, random_state=42, 
                             n_jobs=-1)
best_score = 0
skf = StratifiedKFold(n_splits=5, shuffle=True)
for train_index, test_index in skf.split(X_train, y_train):
    # Decision Trees First
    X_train_cv = np.array(X_train)[train_index, :]
    X_test_cv = np.array(X_train)[test_index, :]
    y_train_cv = np.array(y_train)[train_index]
    y_test_cv = np.array(y_train)[test_index]
    gcv1 = GridSearchCV(rfc, parameters, n_jobs=-1, verbose=1)
    gcv1.fit(X_train_cv, y_train_cv)
    print(gcv1.best_estimator_)
    print(gcv1.best_score_)
    score = gcv1.best_score_
    if score > best_score:
        recorded_train_indexes = train_index
        recorded_test_indexes = test_index
        recorded_best_parameters = gcv1.best_estimator_
        
print(recorded_best_parameters)

In [ ]:
# kNN third
# from sklearn.neighbors import KNeighborsClassifier

# knn_params = {'n_neighbors': range(1, 2)}
# knn = KNeighborsClassifier(n_jobs=-1)

# knn_grid = GridSearchCV(knn, knn_params, cv=skf, n_jobs=-1, verbose=True)
# knn_grid.fit(X_train_last, y_train)
# knn_grid.best_params_, knn_grid.best_score_

Don't expect high cross-val scores! Our dataset is way too small...
For my case, PCA + Decision Trees seems to perform better than worst..

In [ ]:
# Now build your test set;
# Generate a new dataframe for recommended tracks
# Set recommendation limit as half the Playlist Length per track, you may change this as you like
# Check documentation for  recommendations; https://beta.developer.spotify.com/documentation/web-api/reference/browse/get-recommendations/

#rec_tracks = []
#for i in playlist_df['id'].values.tolist():
#    rec_tracks += sp.recommendations(seed_tracks=[i], limit=3)['tracks'];
#
#rec_track_ids = []
#rec_track_names = []
#for i in rec_tracks:
#    rec_track_ids.append(i['id'])
#    rec_track_names.append(i['name'])
#
#rec_features = []
#for i in range(0,len(rec_track_ids)): 
#    rec_audio_features = sp.audio_features(rec_track_ids[i])
#    for track in rec_audio_features:
#        rec_features.append(track)
#        
#rec_playlist_df = pd.DataFrame(rec_features, index = rec_track_ids)
#rec_playlist_df.head()

In [ ]:
X_test_names = v.transform(rec_track_names)

In [ ]:
rec_playlist_df=rec_playlist_df[["acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]

In [ ]:
# Make predictions
X_train_set = np.array(X_train)[recorded_train_indexes, :]
X_test_set = np.array(X_train)[recorded_test_indexes, :]
y_train_set = np.array(y_train)[recorded_train_indexes]
y_test_set = np.array(y_train)[recorded_test_indexes]
gcv1.best_estimator_ = recorded_best_parameters
gcv1.best_estimator_.fit(X_train_set, y_train_set)
#rec_playlist_df_scaled = StandardScaler().fit_transform(rec_playlist_df)
#rec_playlist_df_pca = pca1.transform(rec_playlist_df_scaled)
#X_test_last = csr_matrix(hstack([rec_playlist_df_pca, X_test_names]))
y_pred_class = gcv1.best_estimator_.predict(X_test_set)
print(y_pred_class)

In [ ]:
rec_playlist_ratings = y_pred_class
#rec_playlist_df = rec_playlist_df.sort_values('ratings', ascending = False)
#rec_playlist_df = rec_playlist_df.reset_index()
# Pick the top ranking tracks to add your new playlist 9, 10 will work
ten_indexes = []
nine_indexes = []
for i in range(0, len(y_pred_class)):
    if y_pred_class[i] == 10:
        ten_indexes.append(i)
#    if y_pred_class[i] == 9:
#        nine_indexes.append(i)
#recs_to_add = rec_playlist_df[rec_playlist_df['ratings']>=9]['index'].values.tolist()

recomended_song_indexes = []
for i in ten_indexes:
    recomended_song_indexes.append(recorded_indices[0, test_index[i]])
#for i in nine_indexes:
#    recomended_song_indexes.append(recorded_indices[0, test_index[i]])
print(recomended_song_indexes)

In [ ]:
# No ratings of 9 or 10 this case try adding 8's only
#recs_to_add = rec_playlist_df[rec_playlist_df['ratings']==8]['index'].values.tolist()

In [ ]:
# Check what is about to happen :)
for i in recomended_song_indexes:
    print(song_info['id'][i])

In [ ]:
#rec_array = np.array(recs_to_add[:5])

In [ ]:
# Create a new playlist for tracks to add - you may also add these tracks to your source playlist and proceed
playlist_recs = sp.user_playlist_create(username, 
                                        name='CS464 - {}'.format(sourcePlaylist['name']))


In [ ]:
# Add tracks to the new playlist
tracks = {}
for i in recomended_song_indexes:
    tracks = {song_info['id'][i]}
    print(tracks)
    sp.playlist_add_items(playlist_recs['id'], tracks, position=None)
#     sp.user_playlist_add_tracks(username, playlist_recs['id'], i)

In [ ]:
# This performed really well for my case, and I am pretty satisfied with the resulting playlist -more than the default recommendations